# Advanced Lanes Detection

## Camera Calibration

In [60]:
# Import the necessary libraries
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import glob
from moviepy.editor import VideoFileClip
#%matplotlib inline

In [61]:
# Read in and make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

In [62]:
# The function cal_undistort takes an image, object points, and image points
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [63]:
# This function makes sure that each processed image is saved in the 
# appropriate folder 
def save_img(img, folder, fname, stage_name, col_map):
    fname = fname.split('/')[1]
    fname = fname.split('.')[0]
    new_filename = fname + "_" + stage_name + '.jpg'    
    mpimg.imsave(folder + "/" + new_filename, img,cmap=col_map)

In [64]:
# Arrays to store object points and image points from all the images

objpoints = [] # 3D points in real world space
imgpoints = [] # 2D points in image plane 

# Prepare object points 
objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) # x,y coordinates

# Create the undistorted_images directory within the camera_cal directory
if not os.path.exists("camera_cal/undistorted_images"):
    os.makedirs("camera_cal/undistorted_images")

for fname in images:
    # read in each image
    img = mpimg.imread(fname)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    
    # If corners are found, add object and image points 
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
        
        # draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
        # get the undistorted version of the calibration image
        undistorted = cal_undistort(img, objpoints, imgpoints)
        
        save_img(undistorted, "camera_cal/undistorted_images", fname, "undist", col_map = 'jet')

## Computer Vision Pipeline 

In [65]:
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(warped, window_width, window_height, margin):
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    past_centroids = []
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))
        past_centroids.append(np.average(window_centroids[-3:], axis = 0))

    return past_centroids


In [66]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [67]:
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    dir_grad = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(dir_grad)
    binary_output[(dir_grad > thresh[0]) & (dir_grad < thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

In [68]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    # Apply threshold
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

In [69]:
previous_frames = []
previous_radii = []
previous_positions = []

In [70]:
# Create the undistorted directory 
if not os.path.exists("output_images/undistorted"):
    os.makedirs("output_images/undistorted")
# Create the binary directory 
if not os.path.exists("output_images/binary"):
    os.makedirs("output_images/binary")
# Create the warped directory 
if not os.path.exists("output_images/warped"):
    os.makedirs("output_images/warped")
# Create the lane_pixels directory 
if not os.path.exists("output_images/lane_pixels"):
    os.makedirs("output_images/lane_pixels")
# Create the polynomial directory 
if not os.path.exists("output_images/polynomial"):
    os.makedirs("output_images/polynomial")

    
# This function processes each individual image coming from the video stream 
# and estimates where the lane lines are
def image_pipeline(img, fname):
    undistorted = cal_undistort(img, objpoints, imgpoints)
    if fname != "None":
        save_img(undistorted, "output_images/undistorted", fname, "undistorted", col_map = 'jet')
        
    s_thresh=(90, 255) 
    v_thresh=(200, 255) 
    sx_thresh=(20, 255) 
    sy_thresh=(10, 255) 
    
    gradx = abs_sobel_thresh(undistorted, orient='x', thresh=sx_thresh)
    grady = abs_sobel_thresh(undistorted, orient='y', thresh=sy_thresh)
    hls = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HLS).astype(np.float)
    s_channel = hls[:,:,2]
    hsv = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HSV).astype(np.float)
    v_channel = hsv[:,:,2]
    
    # Threshold color channel
    s_binary = np.zeros_like(undistorted[:,:,0])
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1]) & (v_channel >= v_thresh[0]) & (v_channel <= v_thresh[1])] = 1

    binary_final = np.zeros_like(s_channel)
    binary_final[((gradx == 1) & (grady == 1) | (s_binary == 1))] = 255
    if fname != "None":
        save_img(binary_final, "output_images/binary", fname, "binary", col_map = 'gray')
    
    # Apply a birds-eye view's perspective transform
    src = np.float32([[264, 678],[1042, 678],[686, 452],[596, 452]])
    dst = np.float32([[320, 720],[960, 720],[960, 0],[320, 0]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (binary_final.shape[1],binary_final.shape[0])
    warped = cv2.warpPerspective(binary_final, M, img_size, flags=cv2.INTER_LINEAR)
    if fname != "None":
        save_img(warped, "output_images/warped", fname, "warped", col_map = 'gray')
    
    # Apply a sliding window search
    # window settings
    window_width = 30 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 30 # How much to slide left and right for searching
    window_centroids = find_window_centroids(warped, window_width, window_height, margin)
    # If we found any window centers
    if len(window_centroids) > 0:
        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channle 
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)
    if fname != "None":
        save_img(output, "output_images/lane_pixels", fname, "lane_pixels", col_map = 'jet')
    
    # Apply polynomial fits to the left and right lanes
    if len(window_centroids) > 0:
        leftx = []
        lefty = []
        for (x,y), value in np.ndenumerate(l_points):
            if l_points[x,y] == 255:
                leftx.append(y)
                lefty.append(x)
        rightx = []
        righty = []
        for (x,y), value in np.ndenumerate(r_points):
            if r_points[x,y] == 255:
                rightx.append(y)
                righty.append(x) 
    
        ploty = np.linspace(0, 719, num=720)
        left_fit = np.polyfit(lefty, leftx, 2) 
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fit = np.polyfit(righty, rightx, 2)
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        mark_size = 3
        fig = plt.figure()
        plt.plot(leftx, lefty, 'o', color='red', markersize=mark_size)
        plt.plot(rightx, righty, 'o', color='blue', markersize=mark_size)
        plt.xlim(0, 1280)
        plt.ylim(0, 720)
        plt.plot(left_fitx, ploty, color='green', linewidth=3)
        plt.plot(right_fitx, ploty, color='green', linewidth=3)
        plt.gca().invert_yaxis()
        if fname != "None":
            fname_n = fname
            fname_n = fname_n.split('/')[1]
            fname_n = fname_n.split('.')[0]
            new_filename = fname_n + "_" + "poly" + '.jpg'  
            fig.savefig('output_images/polynomial/' + new_filename)
        
        # Calculate the curvature of the road and the position of the vehicle with respect to 
        # the center of the lane
    
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension

        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(np.asarray(lefty)*ym_per_pix, np.asarray(leftx)*xm_per_pix, 2)
        right_fit_cr = np.polyfit(np.asarray(righty)*ym_per_pix, np.asarray(rightx)*xm_per_pix, 2)
        # Calculate the radii of curvature
        y_bottom_left = np.max(lefty)
        y_bottom_right = np.max(righty)
        left_curverad = ((1 + (2*left_fit_cr[0]*y_bottom_left*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_bottom_right*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        # Now our radius of curvature is in meters
        #print(left_curverad, 'm', right_curverad, 'm')
        avg_curve_rad = (left_curverad + right_curverad)/2
        if fname == "None":
            previous_radii.append(avg_curve_rad)
            avg_curve_rad = np.average(previous_radii[-10:])
        radius = "Radius of Curvature = " + str(avg_curve_rad) + " m"
        center_lane = np.average(window_centroids[0])
        center_offset = (center_lane - warped.shape[1]/2)*xm_per_pix
        if fname == "None":
            previous_positions.append(center_offset)
            center_offset = np.average(previous_positions[-10:])
        if center_offset < 0:
            position = "left"
        else:
            position = "right"
        pos_vehicle = "Vehicle is " + str(center_offset) + " m " + position + " of center"
        if fname != "None":
            print(fname)
            print(radius)
            print(pos_vehicle)
        
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
        left_top = left_fitx[0]
        left_bottom = left_fitx[-1]
        right_top = right_fitx[0]
        right_bottom = right_fitx[-1]
        
        if len(previous_frames) > 0: 
            if (right_top - left_top) < 610 or (right_bottom - left_bottom) < 610 or (right_top - left_top) > 850 or (right_bottom - left_bottom) > 850:
                color_warp = previous_frames[-1]
            else:
                previous_frames.append(color_warp)
        else:
            previous_frames.append(color_warp)
                
        # Check that the new detected lane is similar to the one detected in the 
        # previous frame
        if fname == "None":
            if not previous_frames:
                previous_frames.append(color_warp)
            else:
                color_warp_gray = cv2.cvtColor(color_warp, cv2.COLOR_RGB2GRAY) 
                previous_gray = cv2.cvtColor(previous_frames[-1], cv2.COLOR_RGB2GRAY)  
                non_similarity = cv2.matchShapes(color_warp_gray,previous_gray,1,0.0)
                if non_similarity > 0.002: 
                    color_warp = previous_frames[-1]
                else:
                    previous_frames.append(color_warp)
        
        # Inverse perspective transform
        Minv = cv2.getPerspectiveTransform(dst, src)

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (binary_final.shape[1], binary_final.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
        cv2.putText(result,radius,(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2)
        cv2.putText(result,pos_vehicle,(50,100) , cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2)
        if fname != "None":
            save_img(result, "output_images", fname, "final", col_map = 'jet')
        plt.imshow(result)
        
    return result

## Test Images

In [71]:
# Create the output_images directory 
if not os.path.exists("output_images"):
    os.makedirs("output_images")
    
# Read in and make a list of the test images
test_images = glob.glob('test_images/*.jpg')

for fname in test_images:
    # read in each image
    img = mpimg.imread(fname)
    
    result = image_pipeline(img, fname)

test_images/straight_lines1.jpg
Radius of Curvature = 12198.2221233 m
Vehicle is -0.00660714285714 m left of center
test_images/straight_lines2.jpg
Radius of Curvature = 11216.48909 m
Vehicle is -0.0118928571429 m left of center
test_images/test1.jpg
Radius of Curvature = 2466.16234612 m
Vehicle is 0.196892857143 m right of center
test_images/test2.jpg
Radius of Curvature = 572.28853645 m
Vehicle is 0.262964285714 m right of center
test_images/test3.jpg
Radius of Curvature = 651.133279375 m
Vehicle is 0.215392857143 m right of center
test_images/test4.jpg
Radius of Curvature = 6688.5064407 m
Vehicle is 0.288071428571 m right of center
test_images/test5.jpg
Radius of Curvature = 641.156354907 m
Vehicle is 0.0224642857143 m right of center
test_images/test6.jpg
Radius of Curvature = 637.855729988 m
Vehicle is 0.354142857143 m right of center


## Test on Videos 

In [72]:
previous_frames = []
previous_radii = []
previous_positions = []

In [73]:
def process_image(image):
    result = image_pipeline(image, "None")
    return result

In [74]:
project_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [59:23<00:02,  2.61s/it] 


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 56min 17s, sys: 3min 19s, total: 59min 36s
Wall time: 59min 24s


In [ ]:
challenge_video_output = 'challenge_video_output.mp4'
clip2 = VideoFileClip("challenge_video.mp4")
challenge_video_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_video_clip.write_videofile(challenge_video_output, audio=False)

In [ ]:
harder_challenge_video_output = 'harder_challenge_video_output.mp4'
clip3 = VideoFileClip("harder_challenge_video.mp4")
harder_challenge_video_clip = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time harder_challenge_video_clip.write_videofile(harder_challenge_video_output, audio=False)